<a href="https://colab.research.google.com/github/Mohittt17/Responsible-AI-in-Healthcare-Clinical-Decision-Support-/blob/main/(CDS)_Tiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PneumoniaCDS(nn.Module):
    def __init__(self):
        super(PneumoniaCDS, self).__init__()

        # -------- Feature Extraction Backbone --------
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Dropout(0.4)
        )

        # -------- Classification Head --------
        self.fc1 = nn.Linear(256, 128)
        self.classifier = nn.Linear(128, 2)

        # -------- Risk Assessment Head --------
        self.risk_fc = nn.Linear(256, 64)
        self.risk_output = nn.Linear(64, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        x = x.view(x.size(0), -1)

        # Classification
        cls = F.relu(self.fc1(x))
        cls_out = self.classifier(cls)

        # Risk head
        risk = F.relu(self.risk_fc(x))
        risk_out = F.softmax(self.risk_output(risk), dim=1)

        return cls_out, risk_out

In [2]:
def assign_cds_tier(class_logits, risk_probs):
    confidence = torch.max(F.softmax(class_logits, dim=1)).item()
    risk_level = torch.argmax(risk_probs).item()

    # 0 = Low, 1 = Medium, 2 = High

    if confidence >= 0.8 and risk_level == 0:
        return "Tier 1 - Fully Automated"

    elif confidence >= 0.6 and risk_level in [0,1]:
        return "Tier 2 - AI Assisted"

    else:
        return "Tier 3 - Physician Review Required"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PneumoniaCDS().to(device)

criterion_class = nn.CrossEntropyLoss()
criterion_risk = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [6]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Placeholder: In a real scenario, you would load your actual dataset here.
# For demonstration, creating a dummy dataset.
num_samples = 100
image_size = 224
batch_size = 32
num_classes = 2 # Assuming 2 classes for classification

dummy_images = torch.randn(num_samples, 3, image_size, image_size)
dummy_labels = torch.randint(0, num_classes, (num_samples,))

dummy_dataset = TensorDataset(dummy_images, dummy_labels)
train_loader = DataLoader(dummy_dataset, batch_size=batch_size, shuffle=True)

for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    class_out, risk_out = model(images)

    loss_class = criterion_class(class_out, labels)

    # Dummy risk labels (can be confidence-based)
    risk_labels = torch.zeros_like(labels)
    loss_risk = criterion_risk(risk_out, risk_labels)

    loss = loss_class + 0.3 * loss_risk
    loss.backward()
    optimizer.step()

In [8]:
model.eval()

# Creating a dummy test_loader for demonstration purposes
num_test_samples = 20
dummy_test_images = torch.randn(num_test_samples, 3, image_size, image_size)
dummy_test_labels = torch.randint(0, num_classes, (num_test_samples,))
dummy_test_dataset = TensorDataset(dummy_test_images, dummy_test_labels)
test_loader = DataLoader(dummy_test_dataset, batch_size=1, shuffle=False)

with torch.no_grad():
    image, label = next(iter(test_loader))
    image = image.to(device)

    class_out, risk_out = model(image)

    probs = torch.softmax(class_out, dim=1)
    confidence, predicted = torch.max(probs, 1)

    risk_level = torch.argmax(risk_out, dim=1)

    class_names = ["Normal", "Pneumonia"]
    risk_names = ["Low", "Medium", "High"]

    print("Prediction:", class_names[predicted.item()])
    print("Confidence:", round(confidence.item() * 100, 2), "%")
    print("Risk Level:", risk_names[risk_level.item()])
    print("CDS Tier:", assign_cds_tier(class_out, risk_out))

Prediction: Pneumonia
Confidence: 50.07 %
Risk Level: Low
CDS Tier: Tier 3 - Physician Review Required
